<a href="https://colab.research.google.com/github/A5HU705H/A5HU705H/blob/main/%24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! unzip /content/drive/MyDrive/currency-prediction-challenge.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/train/2270.jpg     
  inflating: data/train/2271.jpg     
  inflating: data/train/2272.jpg     
  inflating: data/train/2273.jpg     
  inflating: data/train/2274.jpg     
  inflating: data/train/2275.jpg     
  inflating: data/train/2276.jpg     
  inflating: data/train/2277.jpg     
  inflating: data/train/2278.jpg     
  inflating: data/train/2279.jpg     
  inflating: data/train/228.jpg      
  inflating: data/train/2280.jpg     
  inflating: data/train/2281.jpg     
  inflating: data/train/2282.jpg     
  inflating: data/train/2283.jpg     
  inflating: data/train/2284.jpg     
  inflating: data/train/2285.jpg     
  inflating: data/train/2286.jpg     
  inflating: data/train/2287.jpg     
  inflating: data/train/2288.jpg     
  inflating: data/train/2289.jpg     
  inflating: data/train/229.jpg      
  inflating: data/train/2290.jpg     
  inflating: data/train/2291.jpg     
  inflating: data/train/2292.jpg     

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2 as cv
import os
import json
import shutil

In [5]:
X_train= pd.read_csv('train.csv')
X_test= pd.read_csv('test.csv')

In [6]:
labelling= open('cat_to_name.json')
labels= labelling.read()
labels_dict= json.loads(labels)

Currency= X_train['currency'].copy()
Country= X_train['country'].copy()
X_train['value']=X_train['value'].str.cat(Currency, sep= ",")
X_train['value']=X_train['value'].str.cat(Country, sep= ",")

Labels_dict = labels_dict
labels_dict = { labels_dict[k]:k for k in labels_dict}
X_train['label']= X_train['value'].map(labels_dict)
x_train = pd.DataFrame([X_train['name'], X_train['label']]).transpose()

from sklearn.utils import shuffle
x_train= shuffle(x_train)

In [7]:
! mkdir 'Train'
! mkdir 'Test'

In [8]:
from os import path
for i in Labels_dict:
  Path1= path.join('/content/Train', i.zfill(4))
  Path2= path.join('/content/Test', i.zfill(4))
  os.mkdir(Path1)
  os.mkdir(Path2)

In [9]:
dictionary= dict(x_train.values)
for j in dictionary:
  src= path.join('/content/data/train', j)
  dst= path.join('/content/Train', (dictionary.get(j)).zfill(4))
  shutil.copy(src, dst)

In [74]:
from keras.preprocessing.image import ImageDataGenerator 
train_datagen=ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=40,
    width_shift_range=0.0,
    height_shift_range=0.0,
    validation_split=0.2,
    horizontal_flip=True,rescale=1./255)
training_set=train_datagen.flow_from_directory('/content/Train',target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=64, subset='training')
validation_set=train_datagen.flow_from_directory('/content/Train',target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=64, subset='validation')

Found 5248 images belonging to 211 classes.
Found 1165 images belonging to 211 classes.


In [75]:
from keras.applications.inception_v3 import InceptionV3
pre_trained_model= InceptionV3(input_shape= (224,224,3), include_top=False, weights= 'imagenet')
for layer in pre_trained_model.layers:
  layer.trainable= False

x= tf.keras.layers.Flatten()(pre_trained_model.output)
x= tf.keras.layers.Dense(1024, activation='relu')(x)
x= tf.keras.layers.Dropout(0.3)(x)
x= tf.keras.layers.Dense(211, activation='softmax')(x)

model= tf.keras.Model(pre_trained_model.input, x)

In [76]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay= 1e-8)
model.compile(loss= 'categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

es=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=5,restore_best_weights=True,verbose=1)
mckpt=tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/incep_coin_pred_model.h5',verbose=1,save_best_only=True,save_weights_only=False,monitor='val_accuracy')
callbacks=[es,mckpt]

model.fit(x=training_set,validation_data=validation_set,epochs=30)



Epoch 1/30
82/82 [==============================] - 110s 1s/step - loss: 4.9896 - accuracy: 0.0949 - val_loss: 3.9621 - val_accuracy: 0.2738
Epoch 2/30
82/82 [==============================] - 89s 1s/step - loss: 3.5848 - accuracy: 0.2929 - val_loss: 2.9454 - val_accuracy: 0.4421
Epoch 3/30
82/82 [==============================] - 89s 1s/step - loss: 2.7090 - accuracy: 0.4394 - val_loss: 2.3524 - val_accuracy: 0.5356
Epoch 4/30
82/82 [==============================] - 89s 1s/step - loss: 2.1628 - accuracy: 0.5404 - val_loss: 2.0250 - val_accuracy: 0.6000
Epoch 5/30
82/82 [==============================] - 89s 1s/step - loss: 1.7457 - accuracy: 0.6164 - val_loss: 1.8021 - val_accuracy: 0.6395
Epoch 6/30
82/82 [==============================] - 88s 1s/step - loss: 1.4744 - accuracy: 0.6723 - val_loss: 1.6649 - val_accuracy: 0.6601
Epoch 7/30
82/82 [==============================] - 88s 1s/step - loss: 1.2837 - accuracy: 0.7130 - val_loss: 1.5753 - val_accuracy: 0.6687
Epoch 8/30
82/82 [=

KeyboardInterrupt: ignored

In [77]:
model.save_weights('/content/drive/MyDrive/inception2')
model.save('/content/drive/MyDrive/inception2.h5')

In [81]:
X_test= pd.read_csv('test.csv')
label= []
for i in X_test['name']:
  Path= path.join('/content/data/test', i)
  img = tf.io.read_file(Path)
  tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
  tensor = tf.image.resize(tensor, [224, 224])
  input_tensor = tf.expand_dims(tensor, axis=0)
  result=model.predict(input_tensor)
  label.append(np.argmax(result[0,:])+1)


In [82]:
print(label)

[1, 191, 11, 143, 48, 179, 135, 104, 173, 124, 23, 151, 103, 126, 45, 193, 91, 106, 144, 112, 89, 69, 73, 97, 128, 6, 61, 85, 36, 131, 104, 54, 101, 113, 73, 153, 171, 50, 54, 35, 124, 181, 8, 29, 34, 88, 87, 180, 97, 119, 100, 51, 140, 43, 86, 200, 28, 129, 209, 77, 78, 159, 138, 174, 48, 118, 92, 52, 53, 198, 171, 192, 19, 197, 62, 183, 181, 160, 140, 40, 126, 21, 49, 92, 146, 104, 128, 50, 202, 35, 40, 192, 101, 83, 113, 66, 1, 194, 190, 204, 104, 60, 82, 130, 167, 9, 8, 197, 68, 49, 39, 211, 186, 81, 56, 4, 84, 13, 15, 140, 32, 43, 187, 126, 198, 143, 199, 143, 2, 81, 210, 93, 60, 50, 165, 79, 99, 205, 133, 180, 82, 41, 169, 169, 58, 15, 67, 37, 20, 184, 25, 195, 154, 127, 132, 14, 47, 33, 146, 30, 156, 103, 70, 135, 192, 46, 88, 160, 77, 156, 64, 25, 12, 76, 159, 44, 154, 107, 152, 192, 36, 100, 202, 193, 158, 38, 206, 83, 156, 208, 136, 5, 79, 94, 144, 178, 7, 52, 146, 96, 199, 22, 28, 166, 202, 5, 19, 119, 167, 74, 32, 76, 63, 82, 94, 151, 132, 26, 119, 208, 148, 161, 135, 21, 3

In [83]:
X_test.insert(1, 'label', label, True)

In [84]:
print(X_test)

        name  label
0      1.jpg      1
1      2.jpg    191
2      3.jpg     11
3      4.jpg    143
4      5.jpg     48
..       ...    ...
839  840.jpg    171
840  841.jpg     53
841  842.jpg    111
842  843.jpg     33
843  844.jpg     94

[844 rows x 2 columns]


In [85]:
X_test.to_csv('/content/drive/MyDrive/submission.csv', index= False)